In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import moscot
from anndata import AnnData
import numpy as np
from moscot.backends.ott import FGWSolver, SinkhornSolver, GWSolver, LRSinkhornSolver
from moscot.solvers._data import TaggedArray, Tag
import matplotlib.pyplot as plt
import jax.numpy as jnp
from typing import *
import pandas as pd
import networkx as nx
import jax.numpy as jnp
from moscot.problems.time._lineage import TemporalProblem
import os
import wot

In [3]:
import ott
import ot

In [4]:
data_path = "/home/icb/dominik.klein/data/wot"


VAR_GENE_DS_PATH = os.path.join(data_path, 'ExprMatrix.var.genes.h5ad')
CELL_DAYS_PATH = os.path.join(data_path, 'cell_days.txt')
SERUM_CELL_IDS_PATH = os.path.join(data_path, 'serum_cell_ids.txt')
CELL_GROWTH_PATH = os.path.join(data_path, 'growth_gs_init.txt')

In [5]:
adata = wot.io.read_dataset(VAR_GENE_DS_PATH, obs=[CELL_DAYS_PATH, CELL_GROWTH_PATH], obs_filter=SERUM_CELL_IDS_PATH)
adata.shape

(175472, 1479)

In [6]:
days = list(adata.obs.day.unique())
days.pop(-1) # remove nan

nan

In [7]:
xs = []
for i in range(len(days)-1):
    print(i)
    c = ot.dist(adata[adata.obs.day==days[i]].X.todense(), adata[adata.obs.day==days[i+1]].X.todense())
    c /= np.median(np.array(c))
    adata.uns["cost_matrix_{}".format(i)] = c
    xs.append({"attr": "uns", "key": "cost_matrix_{}".format(i), "tag": Tag.COST_MATRIX})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [8]:
xs_reduced = xs[:3]
len(xs_reduced)

3

## Run WOT

In [11]:
ot_model = wot.ot.OTModel(adata,epsilon = 0.05, lambda1 = 1,lambda2 = 50) 

In [12]:
tmap_annotated = ot_model.compute_transport_map(11,11.5)

## Run Moscot

In [74]:
tp = TemporalProblem(adata, solver=SinkhornSolver(jit=False))

In [75]:
adata.obs.day = adata.obs.day.astype('category')

In [77]:
tp.prepare("day", subset=[0.0, 0.5, 1.0], x=xs, a_marg={"attr": "obs", "key":"cell_growth_rate"}, n_growth_rates_estimates=3)

In [78]:
lambda_1 = 1.0
lambda_2 = 50.0
eps = 5.0
tp.solve(eps=eps, tau_a=lambda_1/(lambda_1+eps), tau_b=lambda_2/(lambda_2+eps))

2022-02-01 17:07:19.661069: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-02-01 17:07:20.338034: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-02-01 17:07:20.338598: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to 

In [61]:
tp._a_marg_container["g_g1"].isnull()

index
D0_Dox_C1_AAACCTGAGTATCTCG-1         False
D0_Dox_C1_AAACCTGCAAGCGAGT-1         False
D0_Dox_C1_AAACCTGGTCTCTTAT-1         False
D0_Dox_C1_AAACCTGTCAACACTG-1         False
D0_Dox_C1_AAACCTGTCGTTGCCT-1         False
                                     ...  
DiPSC_serum_C2_TTTGTCAGTTCCCGAG-1     True
DiPSC_serum_C2_TTTGTCATCACAGGCC-1     True
DiPSC_serum_C2_TTTGTCATCCTTTACA-1     True
DiPSC_serum_C2_TTTGTCATCTTACCGC-1     True
DiPSC_serum_C2_TTTGTCATCTTCGGTC-1     True
Name: g_g1, Length: 175472, dtype: bool

In [79]:
tp._a_marg_container.loc[~tp._a_marg_container["g_g1"].isnull(), :].day.unique()

[0.0, 0.5]
Categories (39, float64): [0.0, 0.5, 1.0, 1.5, ..., 16.5, 17.0, 17.5, 18.0]

In [81]:
tp.adata.obs["g_g1"]

index
D0_Dox_C1_AAACCTGAGTATCTCG-1         8.663000e-07
D0_Dox_C1_AAACCTGCAAGCGAGT-1         9.204264e-07
D0_Dox_C1_AAACCTGGTCTCTTAT-1         9.030643e-07
D0_Dox_C1_AAACCTGTCAACACTG-1         9.134822e-07
D0_Dox_C1_AAACCTGTCGTTGCCT-1         8.971998e-07
                                         ...     
DiPSC_serum_C2_TTTGTCAGTTCCCGAG-1             NaN
DiPSC_serum_C2_TTTGTCATCACAGGCC-1             NaN
DiPSC_serum_C2_TTTGTCATCCTTTACA-1             NaN
DiPSC_serum_C2_TTTGTCATCTTACCGC-1             NaN
DiPSC_serum_C2_TTTGTCATCTTCGGTC-1             NaN
Name: g_g1, Length: 175472, dtype: float64

In [80]:
tp.adata[~tp.adata.obs["g_g3"].isnull()].obs.day.unique()

[0.0, 0.5]
Categories (2, float64): [0.0, 0.5]

## Compare

In [84]:
tmap_moscot = tp.transport_matrix[(11.0, 11.5)]